In [1]:
import pandas as pd
import glob
import os
from functools import reduce

In [ ]:
DATA_DIR = "Datasets/"

paths = glob.glob(os.path.join(DATA_DIR, "lka_*_2020*.csv")) + \
        glob.glob(os.path.join(DATA_DIR, "lka_*_2020*.xlsx"))

LAYER_MAP = {
    "lka_general_2020":"pop_overall",
    "lka_men_2020":"pop_men",
    "lka_women_2020":"pop_women",
    "lka_children_under_five_2020":"pop_0_5",
    "lka_youth_15_24_2020":"pop_15_24",
    "lka_elderly_60_plus_2020":"pop_60_plus",
    "lka_women_of_reproductive_age_15_49_2020":"pop_women_15_49",
}

In [3]:
dfs = []
for fp in paths:
    fname = os.path.basename(fp).split(".")[0]
    if fname not in LAYER_MAP:
        continue
    
    short = LAYER_MAP[fname]
    
    if fp.lower().endswith(".csv"):
        df = pd.read_csv(fp)
    else:
        df = pd.read_excel(fp)
    
    value_col = df.columns[-1]
    
    df = df[["longitude", "latitude", value_col]].rename(columns={value_col: short})
    
    dfs.append(df)

clean = reduce(
    lambda left, right: pd.merge(left, right, on=["longitude", "latitude"], how="outer"),
    dfs
)

clean = clean.drop_duplicates(subset=["longitude", "latitude"])
clean = clean.dropna(subset=["longitude", "latitude"])
clean = clean.fillna(0)

out_path = os.path.join(DATA_DIR, "cleaned_lka_2020_population_layers.csv")
clean.to_csv(out_path, index=False)
print(f"✅ Cleaned data written to {out_path}")

✅ Cleaned data written to Datasets/cleaned_lka_2020_population_layers.csv


In [ ]:
dfs = []
for fp in paths:
    fname = os.path.basename(fp).split(".")[0]
    if fname not in LAYER_MAP:
        continue
    short = LAYER_MAP[fname]
    if fp.lower().endswith(".csv"):
        df = pd.read_csv(fp)
    else:
        df = pd.read_excel(fp)
    value_col = df.columns[-1]
    df = df[["longitude", "latitude", value_col]].rename(columns={value_col: short})
    dfs.append(df)

clean = reduce(
    lambda left, right: pd.merge(left, right, on=["longitude", "latitude"], how="outer"),
    dfs
)

clean.dropna(subset=["longitude", "latitude"], inplace=True)
clean.drop_duplicates(subset=["longitude", "latitude"], inplace=True)

if len(clean) > 50000:
    clean = clean.sample(n=50000, random_state=42).reset_index(drop=True)

clean.fillna(0, inplace=True)

output_path = os.path.join(DATA_DIR, "cleaned_lka_2020_subset_50000.csv")
clean.to_csv(output_path, index=False)

print(f"✅ Cleaned and sampled data written to {output_path}")